# Data Source Dashboard

In [1]:
# ========== Bootstrap: ensure pandas is available =========
import importlib, subprocess, sys

def _ensure(pkg_name):
    try:
        importlib.import_module(pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[pkg_name] = importlib.import_module(pkg_name)

_ensure('pandas')
from pathlib import Path
import pandas as pd

catalog = pd.read_csv(Path('..', '..', 'data', 'catalog.csv'))
catalog['path'] = catalog.apply(lambda r: str(Path('data', r['category'], r['source'], r['folder'])), axis=1)
dashboard = catalog[['path', 'last_fetched']].sort_values('path').reset_index(drop=True)
dashboard

,path,last_fetched
0,data\demographics\census\zip-demo-ca,2025-06-18
1,data\economics\fred\A939RX0Q048SBEA,2025-07-07T16:29
2,data\economics\fred\CLVMNACSCAB1GQDE,2025-07-07T16:29
3,data\economics\fred\GDPC1,2025-07-07T16:29
4,data\economics\fred\GFDEBTN,2025-07-07T16:29
...,...,...
57,data\news-world\nypost\news-world-nypost,2025-07-15T19:51
58,data\news-world\nyt\news-world-nyt,2025-07-15T19:51
59,data\news-world\toi\news-world-toi,2025-07-15T19:51
60,data\news-world\wapo\news-world-wapo,2025-07-15T17:52


In [2]:
headlines_path = Path("analysis", "headlines", "latest.csv")
headline_count = sum(1 for _ in open(headlines_path)) - 1
print(f"{headline_count} headlines in {headlines_path}")

FileNotFoundError: [Errno 2] No such file or directory: 'analysis\\headlines\\latest.csv'